# Aquisição de dados do diário oficial 
Site: Inlabs Periodicidade: Diária

Pré requisito de browser: https://sites.google.com/a/chromium.org/chromedriver/downloads - Baixar de acordo com a sua versão do Chrome

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import time
from pathlib import Path
import os
import shutil

# Caminho do chromedriver
PATH_CHROMEDRIVER = "./chromedriver"
# Caminho dos arquivos
PATH_ARQUIVOS = "../../../Data/Raw"
# Caminho pasta Downloads do usuário executor do notebook
PATH_DOWNLOADS = str(os.path.join(Path.home(), "Downloads"))

driver = webdriver.Chrome(executable_path=PATH_CHROMEDRIVER)
driver.get("https://inlabs.in.gov.br/acessar.php")

# Usuário e senha de testes. Em ambiente de produção passar para arquivo criptografado.
EMAIL = "marcelolopes@ons.org.br"
SENHA = "P@ssw0rd1"

# Envia email
driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/form[1]/div[1]/div/input").send_keys(EMAIL)
# Envia senha
driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/form[1]/div[2]/div/input").send_keys(SENHA)
# Clica no botão para logar
driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/form[1]/div[3]/div/input").send_keys(Keys.RETURN)

# Formata a data de hoje para AAAA-MM-DD
dataHoje = datetime.now().strftime("%Y-%m-%d")

# Aguarda link dos arquivos do dia ser renderizado no DOM para acessar os arquivos
try:
    linkDataHoje = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, dataHoje))
    )

    linkDataHoje.click()

    # Tabela com todos os arquivos zip e pdf
    tabela = driver.find_element_by_xpath("/html/body/div[1]/div[2]/table")

    # Âncora dos arquivos com link para download
    arquivos = tabela.find_elements_by_xpath("//div[@class='filename']/a")

    print("Site acessado, baixando arquivos")
    print()

    #download dos arquivos zip
    for arquivo in arquivos:
        if ".zip" in arquivo.text:
            arquivo.click()
            print(f"Arquivo {arquivo.text} baixado.")
    
    print()

    # Aguarda 10s para baixar todos os arquivos completamente
    time.sleep(10)

    # Garantir que diretório esteja vazio antes da cópia
    for filename in os.listdir(PATH_ARQUIVOS):
        file_path = os.path.join(PATH_ARQUIVOS, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Falha ao apagar %s. Razão: %s' % (file_path, e))

    # No linux o chrome não permite a alteração do diretório de download para o home. 
    # Os arquivos devem ser movidos para o diretório onde serão consumidos
    
    arquivos = [arq for arq in os.listdir(PATH_DOWNLOADS) if dataHoje in arq]

    for arquivo in arquivos:
        shutil.copy(PATH_DOWNLOADS + "/" + arquivo, PATH_ARQUIVOS)
        print(f"Arquivo {arquivo} copiado para processamento.")

except:
    print(f"Link para os arquivos de {dataHoje} não encontrado no site.")
finally:
    driver.quit()





        

Site acessado, baixando arquivos

Arquivo 2020-09-08-DO3.zip baixado.
Arquivo 2020-09-08-DO2.zip baixado.
Arquivo 2020-09-08-DO1.zip baixado.

Arquivo 2020-09-08-DO2.zip copiado para processamento.
Arquivo 2020-09-08-DO1.zip copiado para processamento.
Arquivo 2020-09-08-DO3.zip copiado para processamento.


In [2]:
pwd

'/home/marcelo/Documentos/infnet/mit_data_science/Code/Operationalization/applications'